# Project Five 
# Game of Thrones - Season One

## Importing and Cleaning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import gensim
from nltk.tag import pos_tag
import re

Using Theano backend.


In [2]:
pd.set_option('display.max_columns', 125)
pd.set_option('display.max_rows', 125)

In [3]:
df_s1 = pd.read_json('/Users/zoemarkovits/Metis/project_five_got/scrape/season_one.json')

In [4]:
df_s1[['title','script_text']]

,title,script_text
0,[Winter is Coming],[[First scene opens with three Rangers riding ...
1,[The Kingsroad],"[\n, \n, Jorah Mormont:, You need to drink, c..."
2,[Lord Snow],"[\n, \n, King’s Landing Guard:, Welcome, Lord..."
3,"[Cripples, Bastards and Broken Things]","[\n, \n, Old Nan:, The little Lord's been dre..."
4,[The Wolf and The Lion],"[\n, Eddard Stark:, He'd never worn this armo..."
5,[A Golden Crown],"[\n, \n, Eddard Stark:, Your pardon, your Gr..."
6,[You Win Or You Die],"[\n, \n, \nPoor Ned Stark. Brave man, terrible..."
7,[The Pointy End],"[King's Landing, \n, \n, Syrio Forel: , Yah! ..."
8,[Baelor],"[\n, Varys: , You've seen better days, my lor..."
9,[Fire and Blood],"[\n, \n, Yoren: , Look at me. Look at me! Do ..."


In [5]:
df_s1['title'] = df_s1['title'].map(lambda x: str(x)[3:-2])

In [6]:
df_s1.title

0                        Winter is Coming
1                           The Kingsroad
2                               Lord Snow
3    Cripples, Bastards and Broken Things
4                   The Wolf and The Lion
5                          A Golden Crown
6                      You Win Or You Die
7                          The Pointy End
8                                  Baelor
9                          Fire and Blood
Name: title, dtype: object

In [7]:
names1_01 = ['WAYMAR ROYCE','WILL','GARED','ROYCE','JON','SEPTA MORDANE','NED','ROBB','BRAN','JON/ROBB','CASSEL',
             'CATELYN','THEON','JAIME','CERSEI','MAESTER LUWIN','LUWIN','ARYA','SANSA','ROBERT','TYRION','ROS',
             'VISERYS','DAENERYS','MAID','ILLYRIO','BENJEN','JORAH','KHAL DROGO','THE HOUND']

names1_02 = ['Jorah Mormont','Daenerys Targaryen','Doreah','Irri','Viserys Targaryen','Joffrey Baratheon',
             'Tyrion Lannister','Sandor Clegane','Jaime Lannister','Myrcella Baratheon','Cersei Baratheon',
             'Catelyn Stark','Jon Snow','Arya Stark','Eddard Stark','Robb Stark','Robert Baratheon','Benjen Stark',
             'Maester Luwin','Assassin','Jhiqui','Theon Greyjoy','Rodrick Cassel','Mycah','Soldier 1','Soldier 2',
             'Jory Cassel','Soldier']

names1_03 = ['King’s Landing Guard','Eddard Stark','Jory Cassel','Jaime Lannister','Eddard Stark','Varys',
             'Renly Baratheon','Petyr Baelish','Grand Maester Pycelle','Cersei Baratheon','Joffrey Baratheon',
             'Septa Mordane','Arya Stark','Sansa Stark','Old Nan','Bran Stark','Robb Stark','Rodrick Cassel',
             'Catelyn Stark','Guard','Alliser Thorne','Tyrion Lannister','Jeor Mormont','Grenn','Jon Snow',
             'Pyp','Rast','Barristan Selmy','Robert Baratheon','Lancel Lannister','Daenerys Targaryen',
             'Jorah Mormont','Viserys Targaryen','Irri','Rakharo','Benjen Stark','Yoren','Maester Aemon',
             'Jeor Mormont','Khal Drogo','Syrio Forel','Arya Stark']

names1_04 = ['Old Nan','Theon Greyjoy','Bran Stark','Tyrion Lannister','Robb Stark','Maester Luwin','Grenn',
             'Jon Snow','Pyp','Alliser Thorne','Samwell Tarly','Jorah Mormont','Viserys Targaryen',
             'Daenerys Targaryen','Doreah','Septa Mordane','Sansa Stark','Janos Slynt','Eddard Stark','Varys',
             'Renly Baratheon','Petyr Baelish','Grand Maester Pycelle','Arya Stark','Jory Cassel','Hugh of the Vale',
             'Tobho Mott','Gendry','Jaime Lannister','Robert Baratheon','Rast','Irri','Cersei Baratheon','Marillion',
             'Catelyn Stark','Rodrick Cassel','Masha Heddle','Bronn','Yoren','Knight of House Whent',
             'Knight of House Bracken']

names1_05 = ['Eddard Stark','Barristan Selmy','Lancel Lannister','Robert Baratheon','Sansa Stark','Petyr Baelish',
             'Renly Baratheon','Loras Tyrell','Sandor Clegane','Catelyn Stark','Marillion','Tyrion Lannister',
             'Rodrik Cassel','Bronn','Maester Luwin','Bran Stark','Theon Greyjoy','Ros','Arya Stark','Varys',
             'Illyrio Mopatis','Guard 1','Guard 2','Jory Cassel','Yoren','Vardis Egen','Steward',
             'Grand Maester Pycelle','Lysa Arryn','Robin Arryn','Mord','Cersei Baratheon','Mhaegen','Jaime Lannister']

names1_06 = ['Eddard Stark','Cersei Baratheon','Robert Baratheon','Irri','Daenerys Targaryen','Robb Stark',
             'Bran Stark','Theon Greyjoy','Osha','Wildling','Wildling 2','Tyrion Lannister','Arya Stark','Syrio Forel',
             'Jorah Mormont','Viserys Targaryen','Mord','Lysa Arryn','Robin Arryn','Catelyn Stark','Knight 1',
             'Knight 2','Knight 3','Knight 4','Knight 5','Ser Vardis','Lancel Lannister','Renly Baratheon',
             'Barristan Selmy','Man','Grand Maester Pycelle','Petyr Baelish','Beric Dondarrion','Mordane',
             'Sansa Stark','Joffrey Baratheon','Ros','Khal Drogo',]

names1_07 = ['Jaime Lannister','Tywin Lannister','Cersei Lannister','Ned Stark','Ros','Petyr Baelish','Theon Greyjoy',
             'Osha','Maester Luwin','Samwell Tarly','Jon Snow','Night Watch stable boy','Renly Baratheon',
             'Robert Baratheon','Barristan Selmy','Lord Varys','Khal Drogo','Daenerys Stormborn','Jorah Mormont',
             'Wine Merchant','Doreah','Rakharo','Jeor Mormont','Alliser Thorne','Jaremy Rykker','Othell Yarwyck',
             'Maester Aemon','Pypar','Tomard','Messenger','Varly','Lord Varys','Janos Slynt','King Joffrey']

names1_08 = ['Syrio Forel','Steward of House Stark','Septa Mordane','Sansa Stark','Arya Stark','Meryn Trant',
             'Sandor Clegane','Stable Boy','Lord Varys','Ned Stark','Jeor Mormont','Othell Yarwyck','Jon Snow',
             'Samwell Tarly','Ser Alliser','Nights Watch brother','Grand Maester Pycelle','Cersei Lannister',
             'Petyr Baelish','Robb Stark','Robb Stark','Theon Greyjoy','Catelyn Stark','Lysa Arryn','Robin Arryn',
             'Bronn','Tyrion Lannister','Shagga','Daenarys Stormborn','Rakharo','Jorah Mormont','Dothraki Man',
             'Mago','Qotho','Khal Drogo','Mirri Maz Duur','Greatjon Umber','Brann Stark','Rickon Stark','Osha',
             'Hodor','Rodrik Cassel','Kevan Lannister','Tywin Lannister','Jamie Lannister','Messenger','Stark guard',
             'Lannister scout','Barristan Selmy','King Joffrey','King’s Landing Page']

names1_09 = ['Varys','Eddard Stark','Robb Stark','Theon Greyjoy','Catelyn Stark','Greatjon Umber','Everyone',
             'Walder Frey','Stevron Frey','Ryger Rivers','Voices outside','Jeor Mormont','Jon Snow',
             'Unidentified Night’s Watchers','Night’s Watcher #1','Night’s Watcher #2','Pyp','Grenn','All',
             'Samwell Tarly','Maester Aemon','Daenerys Targaryen','Khal Drogo','Cohollo','Qotho','Kevan Lannister',
             'Tywin Lannister','Tyrion Lannister','Bronn','Prostitute','Shae','Jorah Mormont','Mirri Maz Duur',
             'Rakharo','Irri','Rodrik Cassel','Stark bannermen','Jaime Lannister','Arya Stark','King’s Landing Baker',
             'Street Urchin','Populace','Grand Maester Pycelle','Joffrey Baratheon','Sansa Stark','Cersei Baratheon',
             'Yoren']

names1_10 = ['Yoren','Arya Stark','Bran Stark','Osha','Rickon Stark','Maester Luwin','Several Stark Bannermen',
             'Robb Stark','Catelyn Stark','Marillion','Joffrey Baratheon','Sansa Stark','Sandor Clegane',
             'Jonos Bracken','Galbart Glover','Greatjon Umber','Rickard Karstark','Theon Greyjoy','All','Stark Guard',
             'Jaime Lannister','Lancel Lannister','Cersei Lannister','Tywin Lannister','Tyrion Lannister',
             'Leo Lefford','Addam Marbrand','Kevan Lannister','Daenerys Targaryen','Jorah Mormont','Mirri Maz Duur',
             'Samwell Tarly','Jon Snow','Shae','Sam, Pyp, and Grenn','Pyp','Grenn','Grand Maester Pycelle','Ros',
             'Varys','Petyr Baelish','Hot Pie','Lommy Greenhands','Gendry','Jeor Mormont','Rakharo']

### Episode One

In [8]:
#df_s1.script_text[0]

In [9]:
script_text1_01 = df_s1.script_text[0]
print type(script_text1_01)

#script_text1_01

<type 'list'>


In [10]:
script_split = [i.split('\n') for i in script_text1_01]
#script_split

In [11]:
string_script_text = ' '

for i in script_split:
    for e in i:
        string_script_text = string_script_text + ' ' + e

In [12]:
#string_script_text

In [13]:
words = word_tokenize(string_script_text)
#words

In [14]:
sentences = sent_tokenize(string_script_text)
#sentences

In [15]:
dialogue = []

for sentence in sentences:
    if sentence.split(':')[0] in (names1_01):
        dialogue.append(sentence)

In [16]:
#dialogue

In [17]:
type(dialogue)

list

In [18]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s101.pkl', 'wb') as f:
    pickle.dump(dialogue, f)

In [19]:
from collections import defaultdict

speakers = defaultdict(list)
for sentence in dialogue:
    speakers[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers

In [20]:
speakers['BENJEN']

[u' Is he dead yet?',
 u' You got bigger.',
 u' Well, you\u2019re always welcome on the wall.',
 u' Jon\u2026    JON',
 u' The Wall isn\u2019t going anywhere.',
 u' You don\u2019t understand what you\u2019d be giving up.',
 u' You might, if you knew what it meant.',
 u' You at a feast -- It\u2019s like a bear in a trap.',
 u' Of course I did.',
 u' The two he was with are still missing.',
 u' Maybe.',
 u' Robb boy.']

In [21]:
episode_one = pd.DataFrame.from_dict(speakers, orient="index")
print episode_one.shape

(27, 33)


In [22]:
#from numpy import nan
#episode_one.fillna(value=str(0), inplace=True)

In [23]:
#episode_one.info()

In [24]:
episode_one.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
KHAL DROGO,No.,No.,No.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
NED,And which one of you was a marksman at ten?,"In the name of Robert of the House Baratheon,...",King of the Andals and the First Men … JON,Lord of the Seven Kingdoms and protector of t...,The White Walkers have been gone for thousand...,A madman sees what he sees.,There are no mountain lions in these woods.,Tough old beast.,"I’m sorry, Bran.",You will train them yourselves.,"Hey,, hey, hey, hey.","Guarding the North for you, Your Grace.",My queen.,Tell me about Jon Arryn.,We both did.,Aye.,I’m not worthy of the honor.,She was my sister.,The boy I beheaded.,He was talking madness.,A wildling ambush.,Your pardon.,"Yes, the king has honored me with his offer.",I don’t fight in tournaments.,I don’t fight in tournaments because when I f...,I’m a Northman.,The king takes what he wants.,Send him in.,Stay.,What news?,"She’s fresh widowed, Cat.","No, but I’m still better than you.",I hope I’ll serve you well.
TYRION,Mmh.,I did hear something about that.,And the other brother?,There’s the pretty one.,I hear he hates that nickname.,Clever girl.,The gods gave me one blessing.,Should I explain to you the meaning of a clos...,JAIME,Close the door!,Your uncle’s in the Night’s Watch.,Did I offend you?,And Lady Stark is not your mother.,All dwarves are bastards in their fathers’ eyes.,If I get through this without squirting from ...,The greatest in the land.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
JAIME,"As your brother, I feel it’s my duty to warn you",CERSEI,"If he told the king, both our heads would be ...",That’s an honor I can do without.,Don’t get up.,I hear we might be neighbors soon.,I’m sure we’ll have a tournament to celebrate...,No?,Well said.,Are you completely mad?,It’s all right.,I heard you the first time.,Ten.,The things I do for love.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
ILLYRIO,(Greets them in Dothraki.),"Come forward, my dear.",The ceremony is over.,"Trust me, Your Grace.",It won’t be long now.,Soon.,I wouldn’t ask Khal Drogo.,"If Khal Drogo has promised you a crown, you s...",When their omens favor war.,A Dothraki wedding without at least three dea...,"Dragons’ eggs, Daenerys.",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [25]:
def remove_stops(sentence):
    characters = ['?','!','.',',',':',';']
    stoplist = stopwords.words('english') + characters
    try: 
        new_sentence = re.sub(r'[^a-zA-Z ]','', sentence)
        new_sentence = new_sentence.lower()
        new_sentence = new_sentence.split()
        new_sentence = [word for word in new_sentence if word not in stoplist]
        new_sentence = ' '.join(new_sentence)
        return new_sentence
    except:
        return sentence   

In [26]:
test = episode_one[0].apply(remove_stops)
#test

In [27]:
for column in episode_one.columns:
    for x in range(len(episode_one[column])):
        episode_one[column][x] = remove_stops(episode_one[column][x])

In [28]:
#episode_one.info()

In [29]:
episode_one.index = episode_one.index.str.title()

In [30]:
episode_one.shape

(27, 33)

In [54]:
#episode_one.head()

### Episode Two

In [32]:
#df_s1.script_text[1]

In [33]:
script_text1_02 = df_s1.script_text[1]
print type(script_text1_02)

#script_text1_02

<type 'list'>


In [34]:
script_split2 = [i.split('\n') for i in script_text1_02]
#script_split2

In [35]:
string_script_text2 = ' '

for i in script_split2:
    for e in i:
        string_script_text2 = string_script_text2 + ' ' + e

In [36]:
#string_script_text2

In [37]:
words2 = word_tokenize(string_script_text2)
#words2

In [38]:
sentences2 = sent_tokenize(string_script_text2)
#sentences2

In [39]:
dialogue2 = []

for sentence in sentences2:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names1_02):
            dialogue2.append(sentence)

In [40]:
#dialogue2

In [41]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s102.pkl', 'wb') as f:
    pickle.dump(dialogue2, f)

In [43]:
from collections import defaultdict

speakers2 = defaultdict(list)
for sentence in dialogue2:
    speakers2[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers2

In [44]:
speakers2['Arya Stark']

[u'  Watch.',
 u'   Shut up.',
 u'  I think so.',
 u'   I know which end to use.',
 u'   Sansa can keep her sewing needles.',
 u'  What are you doing here?',
 u'  Stop it!',
 u'   Nymeria!',
 u"   You've got to go.",
 u'   Go!',
 u'  No.',
 u"   That's not true!",
 u"   That's not what happened!",
 u'   Liar!',
 u'  Liar!',
 u'   Liar liar liar!',
 u"  Lady wasn't there!"]

In [45]:
episode_two = pd.DataFrame.from_dict(speakers2, orient="index")
print episode_two.shape

(27, 30)


In [46]:
#from numpy import nan
#episode_two.fillna(value=str(0), inplace=True)

In [47]:
#episode_two.info()

In [48]:
episode_two.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Joffrey Baratheon,What good will my sympathies do them?,The boy means nothing to me.,I'm telling mother!,You can't... Tyrion Lannister,My princess can drink as much as she wants.,Your sister?,"A butcher's boy who wants to be a Knight, eh?","I'm your prince, not your lord, and I said ...",And you're not a Knight.,I won't hurt him... Much.,"I'll gut you, you little cunt!",Please don't.,Then go!,Yes it is!,Shut up!,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Cersei Baratheon,What do you mean?,"It's no mercy, letting a child linger in suc...",I still can't believe you're going.,The children don't need to hear your filth.,This is your home.,It was years ago.,How dare you speak to your King in that man...,Your girl and that butcher's boy attacked my...,Joffrey told us what happened.,She's not.,She's as wild as that animal of hers.,Joffrey will bear these scars for the rest ...,And what of the direwolf?,We have another wolf.,Where is the beast?,"Ser Ilyn, do me the honor.",Is this some trick?,None,None,None,None,None,None,None,None,None,None,None,None,None
Robert Baratheon,"What do you say, just you and me on the King...","There were wars to fight, women to marry... ...","There was that one... Oh, what was her name?",Bessie!,That's it.,"Trust me, that's not the worst thing.",Soon enough that child will spread her legs ...,"Oh, it's unspeakable to you?","This Khal Drogo, it's said he has 100,000 m...",There are still those in the Seven Kingdoms...,"Quiet, woman.",Enough!,"Now, child... Tell me what happened.","What would you have me do, whip her through ...",I'd forgot the damned wolf.,So be it.,As you will.,A direwolf's no pet.,None,None,None,None,None,None,None,None,None,None,None,None
Eddard Stark,I have no choice.,You can.,"The next time we see each other, we'll talk ...",I've half a mind to go with you.,You should have asked me 20 years ago.,I recall a few chances.,Bessie.,Wylla.,Nor will I. Robert Baratheon,The worst thing about your coronation...,She's little more than a child.,Tell me we're not speaking of this.,"But you can't get your hands on this one, c...",Even a million Dothraki are no threat to the...,He will not cross.,Where is she?,Who took her?,"Right, get back.",Back!,Are you hurt?,It's all right.,In bed asleep.,Stop it!,Stop!,"Gladly, your Grace.",You can't mean it.,s this your command... your Grace?,No.,The wolf is of the north.,The butcher's boy... You rode him down?
Jaime Lannister,Little brother.,Tell me you're not thinking of taking the bl...,Good man.,"At someone, I mean.",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [49]:
test2 = episode_two[0].apply(remove_stops)
#test2

In [50]:
for column in episode_two.columns:
    for x in range(len(episode_two[column])):
        episode_two[column][x] = remove_stops(episode_two[column][x])

In [51]:
#episode_two.info()

In [52]:
episode_two.shape

(27, 30)

In [53]:
#episode_two.head()

### Episode Three

In [55]:
#df_s1.script_text[2]

In [56]:
script_text1_03 = df_s1.script_text[2]
print type(script_text1_03)

#script_text1_03

<type 'list'>


In [57]:
script_split3 = [i.split('\n') for i in script_text1_03]
#script_split3

In [58]:
string_script_text3 = ' '

for i in script_split3:
    for e in i:
        string_script_text3 = string_script_text3 + ' ' + e

In [59]:
#string_script_text3

In [60]:
words3 = word_tokenize(string_script_text3)
#words3

In [61]:
sentences3 = sent_tokenize(string_script_text3)
#sentences3

In [62]:
dialogue3 = []

for sentence in sentences3:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names1_03):
            dialogue3.append(sentence)

/Users/zoemarkovits/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


In [63]:
#dialogue3

In [64]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s103.pkl', 'wb') as f:
    pickle.dump(dialogue3, f)

In [65]:
from collections import defaultdict

speakers3 = defaultdict(list)
for sentence in dialogue3:
    speakers3[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers3

In [66]:
speakers3['Arya Stark']

[u"   I'm practicing.",
 u'  The Prince.',
 u'  The Hound does whatever the Prince tells him to do.',
 u"  You're a liar.",
 u'  Go away!',
 u'   Mine.',
 u"  I wasn't playing.",
 u"   It's called Needle.",
 u"  Stick 'em with the pointy end.",
 u'  I was trying to learn.',
 u'  I hate them!',
 u'  So was I!',
 u'  But how you can let her marry someone like that?',
 u'  Winter is coming.',
 u"  I don't hate her.",
 u'  I can keep it?',
 u'   Who are you?',
 u"   It's too heavy.",
 u'  What if I drop it?',
 u"  I'm a girl."]

In [67]:
episode_three = pd.DataFrame.from_dict(speakers3, orient="index")
print episode_three.shape

(37, 37)


In [68]:
#from numpy import nan
#episode_three.fillna(value=str(0), inplace=True)

In [69]:
#episode_three.info()

In [70]:
episode_three.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
Joffrey Baratheon,It's ugly.,I'm not like him.,Do I have to marry her?,I don't want to.,We allow the northerners too much power.,I'd double their taxes and command them to ...,Why should every lord command his own men?,I'd crush them.,For me.,I'm not asking.,So you agree...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Benjen Stark,I'm the First Ranger.,The kind I don't want to believe.,You're not going.,"The Night's Watch is a joke to you, is it?",I hope we've provided you with some good st...,Call me what you like.,"You know, my brother once told me that nothi...",You're right.,"Enjoy the capital, brother.",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Petyr Baelish,We are the lords of small matters here.,"Mmm, how much?",I'm telling you the Crown is six million in ...,The Master of Coin finds the money.,As you will.,Cat!,No one will come looking for you here.,A dear friend told me.,Good news?,Is she?,I thought that she'd be safest in here.,Which Lord Tyrion will say was stolen from ...,"I'll try to keep you alive, for her sake.",Don't tell anyone.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Cersei Baratheon,"A King should have scars, you fought off a d...",That's not true.,Yes.,"No, but you will.",How would you handle them?,A royal army?,And if the northerners rebel?,"And these 10,000 northern troops, would they...","But you've just invaded their homeland, ask...",The North cannot be held... not by an outsider.,Everyone who isn't us is an enemy.,He's a child... 10 years old.,Nothing.,What if it comes back to him?,And my husband?,Let me go.,Let me go.,And if you find the proof?,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Robert Baratheon,Who was it?,How'd you do it?,Quick one.,What do you mean it's empty?,Is that what empty means?!,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [71]:
test3 = episode_three[0].apply(remove_stops)
#test3

In [72]:
for column in episode_three.columns:
    for x in range(len(episode_three[column])):
        episode_three[column][x] = remove_stops(episode_three[column][x])

In [73]:
#episode_three.info()

In [74]:
#episode_three.head()

### Episode Four

In [75]:
#df_s1.script_text[3]

In [76]:
script_text1_04 = df_s1.script_text[3]
print type(script_text1_04)

#script_text1_04

<type 'list'>


In [77]:
script_split4 = [i.split('\n') for i in script_text1_04]
#script_split4

In [78]:
string_script_text4 = ' '

for i in script_split4:
    for e in i:
        string_script_text4 = string_script_text4 + ' ' + e

In [79]:
#string_script_text4

In [80]:
words4 = word_tokenize(string_script_text4)
#words4

In [81]:
sentences4 = sent_tokenize(string_script_text4)
#sentences4

In [82]:
dialogue4 = []

for sentence in sentences4:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names1_04):
            dialogue4.append(sentence)

In [83]:
#dialogue4

In [84]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s104.pkl', 'wb') as f:
    pickle.dump(dialogue4, f)

In [85]:
from collections import defaultdict

speakers4 = defaultdict(list)
for sentence in dialogue4:
    speakers4[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers4

In [86]:
speakers4['Arya Stark']

[u'  Syrio says every hurt is a lesson and every lesson makes you better.',
 u"  Now that Bran's awake will he come live with us?",
 u' He wants to be a knight of the King\u2019s Guard.',
 u'  Can I be Lord of a holdfast?',
 u'  Why do they call you Littlefinger?']

In [87]:
episode_four = pd.DataFrame.from_dict(speakers4, orient="index")
print episode_four.shape

(36, 45)


In [88]:
#from numpy import nan
#episode_four.fillna(value=str(0), inplace=True)

In [89]:
#episode_four.info()

In [90]:
episode_four.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
Petyr Baelish,I will?,And every inn in the city is full and the wh...,He never stops.,I promised Cat that I'd help you.,A singularly bad idea.,No.,"The wiser answer was no, my Lord.","I have my observers, as I said, and it's pos...","No, it's quite all right.",Ser Gregor Clegane.,Ser Hugh of the Vale.,Not what you were expecting?,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Yoren,"Aye, my Lord.",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Cersei Baratheon,You're missing your tournament.,I thought we might put what happened on the ...,The only Stark who does.,I might ask the same of you.,You can't change him.,"You're just a soldier, aren't you?",As was I.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Robert Baratheon,I'll bet you smell of blackberry jam!,I've been sitting here for days!,"Yes, yes.",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Rast,"You are in love, Lord Snow.",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [91]:
test4 = episode_four[0].apply(remove_stops)
#test4

In [92]:
for column in episode_four.columns:
    for x in range(len(episode_four[column])):
        episode_four[column][x] = remove_stops(episode_four[column][x])

In [93]:
#episode_four.head()

### Episode Five

In [94]:
#df_s1.script_text[4]

In [95]:
script_text1_05 = df_s1.script_text[4]
print type(script_text1_05)

#script_text1_05

<type 'list'>


In [96]:
script_split5 = [i.split('\n') for i in script_text1_05]
#script_split5

In [97]:
string_script_text5 = ' '

for i in script_split5:
    for e in i:
        string_script_text5 = string_script_text5 + ' ' + e

In [98]:
#string_script_text5

In [99]:
words5 = word_tokenize(string_script_text5)
#words5

In [100]:
sentences5 = sent_tokenize(string_script_text5)
#sentences5

In [101]:
dialogue5 = []

for sentence in sentences5:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names1_05):
            dialogue5.append(sentence)

In [102]:
#dialogue5

In [103]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s105.pkl', 'wb') as f:
    pickle.dump(dialogue5, f)

In [104]:
from collections import defaultdict

speakers5 = defaultdict(list)
for sentence in dialogue5:
    speakers5[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers5

In [105]:
speakers5['Arya Stark']

[u"  I'm not a beggar.",
 u'  I want to see my father.',
 u'  My father is Hand of the King!',
 u'  They said they were going to kill you.',
 u"  I didn't see them, but I think one was fat.",
 u"  I'm not lying!",
 u"  I'm a girl!",
 u'  How many guards does my father have?',
 u"  You wouldn't let anyone kill him, would you?"]

In [106]:
episode_five = pd.DataFrame.from_dict(speakers5, orient="index")
print episode_five.shape

(32, 61)


In [107]:
#from numpy import nan
#episode_five.fillna(value=str(0), inplace=True)

In [108]:
#episode_five.info()

In [109]:
#episode_five.head()

In [110]:
test5 = episode_five[0].apply(remove_stops)
#test5

In [111]:
for column in episode_five.columns:
    for x in range(len(episode_five[column])):
        episode_five[column][x] = remove_stops(episode_five[column][x])

In [112]:
episode_five.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
Petyr Baelish,gold dragons mountain,buy gold dragons,loras knew mare heat,honor quite bit gold,move quietly,look bit lonely today,,im purveyor beauty discretion equally important,desires valid man full purse,strictly speaking thing would accordance kings...,tell,since last saw since last saw,saw eyes,council business,know something,yes,long saw escorting certain foreign dignitary c...,find position simple word king renly baratheon,small council meeting,jorah mormonts slaver traitor,find bed ugly woman best close eyes get,grace went length took leave,return winterfell,youre still come nightfall ill take see last p...,wont take hour,brothels make much better investment ship ive ...,well start,matter,kings hand,whats meaning lannister,lords,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Cersei Baratheon,without hand everything fall pieces,,dothraki dont sail,father wrong,still outnumber,five,want,marriage,every day,well seventeen years quite long time,yes tis,first saying name even private felt like breat...,harm could lyanna starks ghost either us haven...,felt something know,even lost first boy quite actually,doesnt make feel anything,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Robert Baratheon,fat,oh funny,,heard hand,long figures,right right,,anybody,telling cowards would let win,hmm bloody idiot,,stop madness name king,whore pregnant,warned would happen,honor,careful ned,hes right,nothing,dies,youre kings hand lord stark,,im glad could something make happy,suppose tell give job brother jaime,dont know,neat little trick move lips fathers voice comes,lets say viserys targaryen lands forty thousan...,bigger number five one,five one,havent real fight nine years,ah sit seventeen years later holding together,long hate hold thing together,yes,youve never asked,,want know horrible truth,know,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Illyrio Mopatis,knows truth,good war,need time,never,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Robin Arryn,mommy,hes little,cant hurt us,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Six

In [113]:
#df_s1.script_text[5]

In [114]:
script_text1_06 = df_s1.script_text[5]
print type(script_text1_06)

#script_text1_06

<type 'list'>


In [115]:
script_split6 = [i.split('\n') for i in script_text1_06]
#script_split6

In [116]:
string_script_text6 = ' '

for i in script_split6:
    for e in i:
        string_script_text6 = string_script_text6 + ' ' + e

In [117]:
#string_script_text6

In [118]:
words6 = word_tokenize(string_script_text6)
#words6

In [119]:
sentences6 = sent_tokenize(string_script_text6)
#sentences6

In [120]:
dialogue6 = []

for sentence in sentences6:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names1_06):
            dialogue6.append(sentence)

In [121]:
#dialogue6

In [122]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s106.pkl', 'wb') as f:
    pickle.dump(dialogue6, f)

In [123]:
from collections import defaultdict

speakers6 = defaultdict(list)
for sentence in dialogue6:
    speakers6[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers6

In [124]:
speakers6['Arya Stark']

[u'  They killed Jory.',
 u'   Yes -   Syrio Forel',
 u'  I don\u2019t like dolls and k -   Syrio Forel',
 u'   Are you dying because of your leg?',
 u'   You can\u2019t.',
 u'   Can\u2019t we take Syrio back with us?',
 u'  Seven hells.',
 u'  The lion\u2019s not his sigil, idiot.',
 u'   Come on!']

In [125]:
episode_six = pd.DataFrame.from_dict(speakers6, orient="index")
print episode_six.shape

(36, 22)


In [126]:
#from numpy import nan
#episode_six.fillna(value=str(0), inplace=True)

In [127]:
#episode_six.info()

In [128]:
#episode_six.head()

In [129]:
test6 = episode_six[0].apply(remove_stops)
#test6

In [130]:
for column in episode_six.columns:
    for x in range(len(episode_six[column])):
        episode_six[column][x] = remove_stops(episode_six[column][x])

In [131]:
episode_six.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Joffrey Baratheon,lady,youll queen one day fitting look part,youre lady,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Petyr Baelish,fish,whisper sounds like someone know,ive heard called tywin lannisters mad dog im s...,bold move lord admirable,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Cersei Baratheon,know wife done,right dare lay hands blood,lord stark returning drunk brothel men attacke...,took king,hes attacked brother abducted,shall wear like badge honor,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Robert Baratheon,whod thought,oh shut mouths,quiet woman,hold tongue,wear silence ill honor,see,tell wife return little shit imp kings landing,im half kingdom debt bloody father,piss,killing things clears head,oh saying,,enemy right open vicious like sending bloody i...,exhilarating yes,think,ever make eight barristan,laughs days,None,None,None,None,None
Khal Drogo,speaks dothraki motions side hut jorah mormont,king,speaks dothraki viserys targaryen,crown king,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Seven

In [132]:
#df_s1.script_text[6]

In [133]:
script_text1_07 = df_s1.script_text[6]
print type(script_text1_07)

#script_text1_07

<type 'list'>


In [134]:
script_split7 = [i.split('\n') for i in script_text1_07]
#script_split7

In [135]:
string_script_text7 = ' '

for i in script_split7:
    for e in i:
        string_script_text7 = string_script_text7 + ' ' + e

In [136]:
#string_script_text7

In [137]:
words7 = word_tokenize(string_script_text7)
#words7

In [138]:
sentences7 = sent_tokenize(string_script_text7)
#sentences7

In [139]:
dialogue7 = []

for sentence in sentences7:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names1_07):
            dialogue7.append(sentence)

In [140]:
#dialogue7

In [141]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s107.pkl', 'wb') as f:
    pickle.dump(dialogue7, f)

In [142]:
from collections import defaultdict

speakers7 = defaultdict(list)
for sentence in dialogue7:
    speakers7[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers7

In [143]:
speakers7['Ned Stark']

[u"   I've had worse, my lady.",
 u'   I know the truth Jon Arryn died for.',
 u'   Has he done this before?',
 u'   Your brother Or your lover?',
 u'   My son saw you with him.',
 u'   With all my heart.',
 u"   And they are all Jaime's.",
 u"   You've always hated him.",
 u"   When the King returns from his hunt, I'll tell him the truth.",
 u"   I've made many mistakes in my life, but that wasn't one of them.",
 u'   You damned fool.',
 u'   I will.',
 u"   I'll, I'll do everything I can to honor your memory.",
 u'   Give him milk of the poppy.',
 u'   No man could have protected him from himself.',
 u'   His squire?',
 u'   His Grace has had a change of heart concerning Daenerys Targaryen.',
 u'   No.',
 u'   There is no other choice.',
 u'   You promised Catelyn you would help me.']

In [144]:
episode_seven = pd.DataFrame.from_dict(speakers7, orient="index")
print episode_seven.shape

(21, 20)


In [145]:
#from numpy import nan
#episode_seven.fillna(value=str(0), inplace=True)

In [146]:
#episode_seven.info()

In [147]:
#episode_seven.head()

In [148]:
test7 = episode_seven[0].apply(remove_stops)
#test7

In [149]:
for column in episode_seven.columns:
    for x in range(len(episode_seven[column])):
        episode_seven[column][x] = remove_stops(episode_seven[column][x])

In [150]:
episode_seven.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Barristan Selmy,reeling wine,squire kings skin,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Petyr Baelish,,lets start shall,im saving another,stupid saying,really,many years,oh,oh everything dear,stannis war,call,look,None,None,None,None,None,None,None,None,None
Samwell Tarly,miss girls,one blast ranger returning two wildlings three...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Ned Stark,ive worse lady,know truth jon arryn died,done,brother lover,son saw,heart,jaimes,youve always hated,king returns hunt ill tell truth,ive made many mistakes life wasnt one,damned fool,,ill ill everything honor memory,give milk poppy,man could protected,squire,grace change heart concerning daenerys targaryen,,choice,promised catelyn would help
Wine Merchant,virzetha gizikhven,daenerys stormborn,lady westeros,princess,,honorthe honor mine,know many homeland pray return princess,wine khaleesi,would crime drink wine rich without least givi...,princess commands,sweet isnt,,None,None,None,None,None,None,None,None


### Episode Eight

In [151]:
#df_s1.script_text[7]

In [152]:
script_text1_08 = df_s1.script_text[7]
print type(script_text1_08)

#script_text1_08

<type 'list'>


In [153]:
script_split8 = [i.split('\n') for i in script_text1_08]
#script_split8

In [154]:
string_script_text8 = ' '

for i in script_split8:
    for e in i:
        string_script_text8 = string_script_text8 + ' ' + e

In [155]:
#string_script_text8

In [156]:
words8 = word_tokenize(string_script_text8)
#words8

In [157]:
sentences8 = sent_tokenize(string_script_text8)
#sentences8

In [158]:
dialogue8 = []

for sentence in sentences8:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names1_08):
            dialogue8.append(sentence)

/Users/zoemarkovits/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


In [159]:
#dialogue8

In [160]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s108.pkl', 'wb') as f:
    pickle.dump(dialogue8, f)

In [161]:
from collections import defaultdict

speakers8 = defaultdict(list)
for sentence in dialogue8:
    speakers8[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers8

In [162]:
speakers8['Arya Stark']

[u'   You said right, but you went left.',
 u"   Only 'cause you lied.",
 u'   I was so.',
 u"   My father wouldn't send you.",
 u"   And I don't have to go with you if I don't want.",
 u'   Come with me.',
 u'   Not today.',
 u'   Not today.',
 u'   Needle.',
 u'   What do you want?',
 u'   Leave me be.',
 u'   Stay away!']

In [163]:
episode_eight = pd.DataFrame.from_dict(speakers8, orient="index")
print episode_eight.shape

(28, 17)


In [164]:
#from numpy import nan
#episode_eight.fillna(value=str(0), inplace=True)

In [166]:
#episode_eight.info()

In [167]:
#episode_eight.head()

In [168]:
test8 = episode_eight[0].apply(remove_stops)
#test8

In [169]:
for column in episode_eight.columns:
    for x in range(len(episode_eight[column])):
        episode_eight[column][x] = remove_stops(episode_eight[column][x])

In [170]:
episode_eight.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
King Joffrey,let father die,come forward lady,said wasnt king,anything else,sweet words moved,None,None,None,None,None,None,None,None,None,None,None,None
Petyr Baelish,girl innocent grace,naked knight apparently,deny fathers crime,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Tyrion Lannister,uncle,sorry disappoint,may present lord father tywin son tytos house ...,differences jaime,starks,sweet sister persuade king imprison dear frien...,sister rules mean,maybe,None,None,None,None,None,None,None,None,None
Samwell Tarly,smell,none,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Sandor Clegane,think sent,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Nine

In [171]:
#df_s1.script_text[8]

In [172]:
script_text1_09 = df_s1.script_text[8]
print type(script_text1_09)

#script_text1_09

<type 'list'>


In [173]:
script_split9 = [i.split('\n') for i in script_text1_09]
#script_split9

In [174]:
string_script_text9 = ' '

for i in script_split9:
    for e in i:
        string_script_text9 = string_script_text9 + ' ' + e

In [175]:
#string_script_text9

In [176]:
words9 = word_tokenize(string_script_text9)
#words9

In [177]:
sentences9 = sent_tokenize(string_script_text9)
#sentences9

In [178]:
dialogue9 = []

for sentence in sentences9:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names1_09):
            dialogue9.append(sentence)

/Users/zoemarkovits/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


In [179]:
#dialogue9

In [180]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s109.pkl', 'wb') as f:
    pickle.dump(dialogue9, f)

In [181]:
from collections import defaultdict

speakers9 = defaultdict(list)
for sentence in dialogue9:
    speakers9[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers9

In [182]:
speakers9['Arya Stark']

[u'   How about a nice fat pigeon?',
 u'   Do you have any stale ones from yesterday?',
 u"   Hey, where's everyone going?",
 u'   Who?',
 u'   Let me go.']

In [183]:
episode_nine = pd.DataFrame.from_dict(speakers9, orient="index")
print episode_nine.shape

(39, 62)


In [184]:
#from numpy import nan
#episode_nine.fillna(value=str(0), inplace=True)

In [185]:
#episode_nine.info()

In [186]:
#episode_nine.head()

In [187]:
test9 = episode_nine[0].apply(remove_stops)
#test9

In [188]:
for column in episode_nine.columns:
    for x in range(len(episode_nine[column])):
        episode_nine[column][x] = remove_stops(episode_nine[column][x])

In [189]:
episode_nine.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61
Joffrey Baratheon,mother wishes let lord eddard join nights watch,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Populace,traitor,traitor,put,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Cersei Baratheon,son madness,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Khal Drogo,horse cohollo,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Yoren,dont look,shut mouth,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Ten

In [190]:
#df_s1.script_text[9]

In [191]:
script_text1_10 = df_s1.script_text[9]
print type(script_text1_10)

#script_text1_10

<type 'list'>


In [192]:
script_split10 = [i.split('\n') for i in script_text1_10]
#script_split10

In [193]:
string_script_text10 = ' '

for i in script_split10:
    for e in i:
        string_script_text10 = string_script_text10 + ' ' + e

In [194]:
#string_script_text10

In [195]:
words10 = word_tokenize(string_script_text10)
#words10

In [196]:
sentences10 = sent_tokenize(string_script_text10)
#sentences10

In [197]:
dialogue10 = []

for sentence in sentences10:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names1_10):
            dialogue10.append(sentence)

In [198]:
#dialogue10

In [199]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s110.pkl', 'wb') as f:
    pickle.dump(dialogue10, f)

In [200]:
from collections import defaultdict

speakers10 = defaultdict(list)
for sentence in dialogue10:
    speakers10[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers10

In [201]:
speakers10['Arya Stark']

[u"   l'm not a boy!",
 u'   Arry.',
 u'     You want it?',
 u'   lt was a gift.',
 u'   Which are you?']

In [202]:
episode_ten = pd.DataFrame.from_dict(speakers10, orient="index")
print episode_ten.shape

(39, 18)


In [203]:
#from numpy import nan
#episode_ten.fillna(value=str(0), inplace=True)

In [204]:
#episode_ten.info()

In [205]:
#episode_ten.head()

In [206]:
test10 = episode_ten[0].apply(remove_stops)
#test10

In [207]:
for column in episode_ten.columns:
    for x in range(len(episode_ten[column])):
        episode_ten[column][x] = remove_stops(episode_ten[column][x])

In [208]:
episode_ten.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Joffrey Baratheon,amusing,tell favor fingers tongue,fingers tongue,good,ser llyn better carry sentence,lm done day,look quite nice,grace,walk,soon youve blood lll put son,ones father,l,well,long pleases,obey,lords,None,None
Petyr Baelish,lts hard simper bow without heads,would friend found sitting,must one men city isnt man,l picture gash,must strange even years man another land despi...,carry whispering one kings ear next,useful talent lm sure youd agree,playing roles,long may reign,None,None,None,None,None,None,None,None,None
Lommy Greenhands,hes got sword one,maybe hes little squire,lets look,take,look,None,None,None,None,None,None,None,None,None,None,None,None,None
Yoren,keep mouth shut boy,youre smart boy youre trying say,youre arry hear,youve long way travel bad company,come sorry sons whores,None,None,None,None,None,None,None,None,None,None,None,None,None
Shae,father said couldnt take anyone kings landing,knew name,said dont bring shae kings landing,ashamed,im funny,None,None,None,None,None,None,None,None,None,None,None,None,None


## Season One

In [209]:
episode_one = episode_one.reset_index()

In [210]:
episode_one_dict = {'Khal Drogo':'Khal Drogo', 
                  'Ned':'Eddard Stark', 
                  'Tyrion':'Tyrion Lannister', 
                  'Jaime':'Jaime Lannister', 
                  'Illyrio':'Illyrio', 
                  'Viserys':'Viserys Targaryen', 
                  'Robert':'Robert Baratheon', 
                  'Daenerys':'Daenerys Targaryen', 
                  'Ros':'Ros', 
                  'Septa Mordane':'Septa Mordane', 
                  'Jon':'Jon Snow', 
                  'Benjen':'Benjen Stark',
                  'Luwin':'Maester Luwin', 
                  'Theon':'Theon Greyjoy', 
                  'Jon/Robb':'Jon/Robb', 
                  'Cersei':'Cersei Baratheon', 
                  'The Hound':'Sandor Clegane', 
                  'Sansa':'Sansa Stark',
                  'Jorah':'Jorah Mormont', 
                  'Gared':'Gared', 
                  'Robb':'Robb Stark', 
                  'Will':'Will', 
                  'Cassel':'Rodrick Cassel', 
                  'Catelyn':'Catelyn Stark', 
                  'Arya':'Arya Stark',
                  'Bran':'Bran Stark', 
                  'Royce':'Royce'}

episode_one['index']=episode_one['index'].replace(episode_one_dict)

In [211]:
episode_one = episode_one.set_index('index')
episode_one.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Khal Drogo,,,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Eddard Stark,one marksman ten,name robert house baratheon first name jon bran,king andals first men jon,lord seven kingdoms protector realm eddard hou...,white walkers gone thousands years,madman sees sees,mountain lions woods,tough old beast,im sorry bran,train,hey hey hey hey,guarding north grace,queen,tell jon arryn,,aye,im worthy honor,sister,boy beheaded,talking madness,wildling ambush,pardon,yes king honored offer,dont fight tournaments,dont fight tournaments fight man real dont wan...,im northman,king takes wants,send,stay,news,shes fresh widowed cat,im still better,hope ill serve well
Tyrion Lannister,mmh,hear something,brother,theres pretty one,hear hates nickname,clever girl,gods gave one blessing,explain meaning closed door whorehouse brother,jaime,close door,uncles nights watch,offend,lady stark mother,dwarves bastards fathers eyes,get without squirting one end miracle,greatest land,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Jaime Lannister,brother feel duty warn,cersei,told king heads would skewered city gates,thats honor without,dont get,hear might neighbors soon,im sure well tournament celebrate new title ac...,,well said,completely mad,right,heard first time,ten,things love,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Illyrio,greets dothraki,come forward dear,ceremony,trust grace,wont long,soon,wouldnt ask khal drogo,khal drogo promised crown shall,omens favor war,dothraki wedding without least three deaths co...,dragons eggs daenerys,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [212]:
print episode_one.shape
print episode_two.shape

(27, 33)
(27, 30)


In [213]:
episode_two.index

Index([u'Joffrey Baratheon', u'Cersei Baratheon', u'Robert Baratheon',
       u'Eddard Stark', u'Jaime Lannister', u'Jhiqui', u'Theon Greyjoy',
       u'Mycah', u'Irri', u'Arya Stark', u'Tyrion Lannister', u'Catelyn Stark',
       u'Soldier', u'Viserys Targaryen', u'Doreah', u'Assassin', u'Soldier 2',
       u'Soldier 1', u'Rodrick Cassel', u'Daenerys Targaryen',
       u'Jorah Mormont', u'Maester Luwin', u'Robb Stark',
       u'Myrcella Baratheon', u'Jory Cassel', u'Sandor Clegane', u'Jon Snow'],
      dtype='object')

In [214]:
season_one = episode_one.join(episode_two, rsuffix="_two", how='outer')
print season_one.shape

(38, 63)


In [215]:
#season_one.head()

In [216]:
#season_one.index

In [217]:
episode_three.shape

(37, 37)

In [219]:
episode_three.index

Index([u'Joffrey Baratheon', u'Benjen Stark', u'Petyr Baelish',
       u'Cersei Baratheon', u'Robert Baratheon', u'Khal Drogo', u'Rast',
       u'Yoren', u'Maester Aemon', u'Tyrion Lannister', u'Pyp',
       u'Eddard Stark', u'Bran Stark', u'Irri', u'Arya Stark',
       u'Jaime Lannister', u'Septa Mordane', u'Old Nan', u'Jeor Mormont',
       u'Alliser Thorne', u'Catelyn Stark', u'Varys', u'Viserys Targaryen',
       u'Sansa Stark', u'Rakharo', u'Barristan Selmy', u'Rodrick Cassel',
       u'Daenerys Targaryen', u'Lancel Lannister', u'Jorah Mormont', u'Grenn',
       u'Guard', u'Robb Stark', u'Renly Baratheon', u'Syrio Forel',
       u'Jory Cassel', u'Jon Snow'],
      dtype='object')

In [220]:
season_one = season_one.join(episode_three, rsuffix="_three", how='outer')
season_one.shape

(54, 100)

In [221]:
#season_one.index

In [222]:
episode_four.shape

(36, 45)

In [223]:
episode_four.index

Index([u'Petyr Baelish', u'Yoren', u'Cersei Baratheon', u'Robert Baratheon',
       u'Rast', u'Janos Slynt', u'Tyrion Lannister', u'Samwell Tarly', u'Pyp',
       u'Theon Greyjoy', u'Eddard Stark', u'Bran Stark', u'Irri',
       u'Arya Stark', u'Jaime Lannister', u'Septa Mordane', u'Tobho Mott',
       u'Gendry', u'Marillion', u'Alliser Thorne', u'Catelyn Stark', u'Varys',
       u'Bronn', u'Viserys Targaryen', u'Doreah', u'Sansa Stark',
       u'Rodrick Cassel', u'Daenerys Targaryen', u'Jorah Mormont', u'Grenn',
       u'Maester Luwin', u'Robb Stark', u'Renly Baratheon', u'Masha Heddle',
       u'Jory Cassel', u'Jon Snow'],
      dtype='object')

In [224]:
season_one = season_one.join(episode_four, rsuffix="_four", how='outer')
season_one.shape

(61, 145)

In [225]:
#season_one.index

In [226]:
episode_five.index

Index([u'Petyr Baelish', u'Cersei Baratheon', u'Robert Baratheon',
       u'Illyrio Mopatis', u'Robin Arryn', u'Yoren', u'Mord',
       u'Tyrion Lannister', u'Theon Greyjoy', u'Vardis Egen', u'Mhaegen',
       u'Eddard Stark', u'Bran Stark', u'Lysa Arryn', u'Arya Stark',
       u'Jaime Lannister', u'Catelyn Stark', u'Marillion', u'Bronn',
       u'Steward', u'Rodrik Cassel', u'Varys', u'Sansa Stark',
       u'Barristan Selmy', u'Lancel Lannister', u'Loras Tyrell',
       u'Maester Luwin', u'Renly Baratheon', u'Guard 1', u'Guard 2', u'Ros',
       u'Jory Cassel'],
      dtype='object')

In [227]:
episode_five.shape

(32, 61)

In [228]:
season_one = season_one.join(episode_five, rsuffix="_five", how='outer')
season_one.shape

(72, 206)

In [229]:
episode_six.index

Index([u'Joffrey Baratheon', u'Petyr Baelish', u'Cersei Baratheon',
       u'Robert Baratheon', u'Khal Drogo', u'Robin Arryn', u'Sansa Stark',
       u'Eddard Stark', u'Mord', u'Tyrion Lannister', u'Wildling',
       u'Theon Greyjoy', u'Ser Vardis', u'Osha', u'Bran Stark', u'Irri',
       u'Lysa Arryn', u'Arya Stark', u'Man', u'Mordane', u'Catelyn Stark',
       u'Viserys Targaryen', u'Knight 5', u'Knight 4', u'Knight 3',
       u'Knight 2', u'Knight 1', u'Barristan Selmy', u'Ros',
       u'Daenerys Targaryen', u'Jorah Mormont', u'Beric Dondarrion',
       u'Robb Stark', u'Renly Baratheon', u'Wildling 2', u'Syrio Forel'],
      dtype='object')

In [230]:
episode_six.shape

(36, 22)

In [231]:
season_one = season_one.join(episode_six, rsuffix="_six", how='outer')
season_one.shape

(84, 228)

In [232]:
episode_seven.index

Index([u'Barristan Selmy', u'Petyr Baelish', u'Samwell Tarly', u'Ned Stark',
       u'Wine Merchant', u'Theon Greyjoy', u'Renly Baratheon',
       u'Tywin Lannister', u'Rakharo', u'Cersei Lannister', u'Osha',
       u'Khal Drogo', u'Jorah Mormont', u'Maester Luwin',
       u'Daenerys Stormborn', u'Doreah', u'Ros', u'Lord Varys',
       u'Robert Baratheon', u'Jon Snow', u'Jaime Lannister'],
      dtype='object')

In [233]:
episode_seven = episode_seven.reset_index()

In [234]:
episode_seven_dict = {'Barristan Selmy':'Barristan Selmy',
                      'Petyr Baelish':'Petyr Baelish',
                      'Samwell Tarly':'Samwell Tarly',
                      'Ned Stark':'Eddard Stark',
                      'Wine Merchant':'Wine Merchant',
                      'Theon Greyjoy':'Theon Greyjoy',
                      'Renly Baratheon':'Renly Baratheon',
                      'Tywin Lannister':'Tywin Lannister',
                      'Rakharo':'Rakharo',
                      'Cersei Lannister':'Cersei Baratheon',
                      'Osha':'Osha',
                      'Khal Drogo':'Khal Drogo',
                      'Jorah Mormont':'Jorah Mormont',
                      'Maester Luwin':'Maester Luwin',
                      'Daenerys Stormborn':'Daenerys Targaryen',
                      'Doreah':'Doreah',
                      'Ros':'Ros',
                      'Lord Varys':'Varys',
                      'Robert Baratheon':'Robert Baratheon',
                      'Jon Snow':'Jon Snow',
                      'Jaime Lannister':'Jaime Lannister'}

episode_seven['index'] = episode_seven['index'].replace(episode_seven_dict)

In [235]:
episode_seven = episode_seven.set_index('index')
episode_seven.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
index,,,,,,,,,,,,,,,,,,,,
Barristan Selmy,reeling wine,squire kings skin,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Petyr Baelish,,lets start shall,im saving another,stupid saying,really,many years,oh,oh everything dear,stannis war,call,look,None,None,None,None,None,None,None,None,None
Samwell Tarly,miss girls,one blast ranger returning two wildlings three...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Eddard Stark,ive worse lady,know truth jon arryn died,done,brother lover,son saw,heart,jaimes,youve always hated,king returns hunt ill tell truth,ive made many mistakes life wasnt one,damned fool,,ill ill everything honor memory,give milk poppy,man could protected,squire,grace change heart concerning daenerys targaryen,,choice,promised catelyn would help
Wine Merchant,virzetha gizikhven,daenerys stormborn,lady westeros,princess,,honorthe honor mine,know many homeland pray return princess,wine khaleesi,would crime drink wine rich without least givi...,princess commands,sweet isnt,,None,None,None,None,None,None,None,None


In [236]:
episode_seven.shape

(21, 20)

In [237]:
season_one = season_one.join(episode_seven, rsuffix="_seven", how='outer')
season_one.shape

(86, 248)

In [238]:
#season_one.index

In [239]:
episode_eight.index

Index([u'King Joffrey', u'Petyr Baelish', u'Tyrion Lannister',
       u'Samwell Tarly', u'Sandor Clegane', u'Tywin Lannister', u'Arya Stark',
       u'Septa Mordane', u'Othell Yarwyck', u'Jeor Mormont', u'Ned Stark',
       u'Stark guard', u'Catelyn Stark', u'Meryn Trant', u'Ser Alliser',
       u'Sansa Stark', u'Shagga', u'Lord Varys', u'Barristan Selmy',
       u'Stable Boy', u'Lannister scout', u'Greatjon Umber',
       u'Cersei Lannister', u'Robb Stark', u'Kevan Lannister', u'Syrio Forel',
       u'Theon Greyjoy', u'Jon Snow'],
      dtype='object')

In [240]:
episode_eight = episode_eight.reset_index()

In [241]:
episode_eight_dict = {'King Joffrey':'Joffrey Baratheon',
                      'Petyr Baelish':'Petyr Baelish',
                      'Tyrion Lannister':'Tyrion Lannister',
                      'Samwell Tarly':'Samwell Tarly',
                      'Sandor Clegane':'Sandor Clegane',
                      'Tywin Lannister':'Tywin Lannister',
                      'Arya Stark':'Arya Stark',
                      'Stark guard':'Stark Guard',
                      'Catelyn Stark':'Catelyn Stark',
                      'Meryn Trant':'Meryn Trant',
                      'Ser Alliser':'Alliser Thorne',
                      'Stable Boy':'Stable Boy',
                      'Lannister scout':'Lannister Scout',
                      'Greatjon Umber':'Greatjon Umber',
                      'Cersei Lannister':'Cersei Baratheon',
                      'Robb Stark':'Robb Stark',
                      'Kevan Lannister':'Kevan Lannister',
                      'Syrio Forel':'Syrio Forel',
                      'Theon Greyjoy':'Theon Greyjoy',
                      'Jon Snow':'Jon Snow'}

episode_eight['index'] = episode_eight['index'].replace(episode_eight_dict)

In [242]:
episode_eight = episode_eight.set_index('index')
episode_eight.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
index,,,,,,,,,,,,,,,,,
Joffrey Baratheon,let father die,come forward lady,said wasnt king,anything else,sweet words moved,None,None,None,None,None,None,None,None,None,None,None,None
Petyr Baelish,girl innocent grace,naked knight apparently,deny fathers crime,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Tyrion Lannister,uncle,sorry disappoint,may present lord father tywin son tytos house ...,differences jaime,starks,sweet sister persuade king imprison dear frien...,sister rules mean,maybe,None,None,None,None,None,None,None,None,None
Samwell Tarly,smell,none,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Sandor Clegane,think sent,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [243]:
episode_eight.shape

(28, 17)

In [244]:
season_one.shape

(86, 248)

In [245]:
season_one = season_one.join(episode_eight, rsuffix="_eight", how='outer')
season_one.shape

(96, 265)

In [246]:
#season_one.index

In [247]:
episode_nine.shape

(39, 62)

In [248]:
season_one = season_one.join(episode_nine, rsuffix="_nine", how='outer')
season_one.shape

(107, 327)

In [249]:
#season_one.index

In [250]:
episode_ten.index

Index([u'Joffrey Baratheon', u'Petyr Baelish', u'Lommy Greenhands', u'Yoren',
       u'Shae', u'Jonos Bracken', u'Tyrion Lannister', u'Samwell Tarly',
       u'Grenn', u'Pyp', u'Theon Greyjoy', u'Stark Guard', u'Tywin Lannister',
       u'Jon Snow', u'Osha', u'Bran Stark', u'Rickon Stark', u'Arya Stark',
       u'Jaime Lannister', u'Jeor Mormont', u'Gendry', u'Catelyn Stark',
       u'Marillion', u'Varys', u'Hot Pie', u'Sansa Stark', u'Leo Lefford',
       u'All', u'Daenerys Targaryen', u'Jorah Mormont', u'Greatjon Umber',
       u'Cersei Lannister', u'Robb Stark', u'Rickard Karstark',
       u'Kevan Lannister', u'Addam Marbrand', u'Ros', u'Sandor Clegane',
       u'Galbart Glover'],
      dtype='object')

In [251]:
episode_ten = episode_ten.reset_index()

In [252]:
episode_ten_dict = {'Joffrey Baratheon':'Joffrey Baratheon',
                    'Petyr Baelish':'Petyr Baelish',
                    'Lommy Greenhands':'Lommy Greenhands',
                    'Yoren':'Yoren',
                    'Shae':'Shae',
                    'Jonos Bracken':'Jonos Bracken',
                    'Tyrion Lannister':'Tyrion Lannister',
                    'Samwell Tarly':'Samwell Tarly',
                    'Grenn':'Grenn',
                    'Pyp':'Pyp',
                    'Theon Greyjoy':'Theon Greyjoy',
                    'Stark Guard':'Stark Guard',
                    'Tywin Lannister':'Tywin Lannister',
                    'Jon Snow':'Jon Snow',
                    'Osha':'Osha',
                    'Bran Stark':'Bran Stark',
                    'Rickon Stark':'Rickon Stark',
                    'Arya Stark':'Arya Stark',
                    'Jaime Lannister':'Jaime Lannister',
                    'Jeor Mormont':'Jeor Mormont',
                    'Gendry':'Gendry',
                    'Catelyn Stark':'Catelyn Stark',
                    'Marillion':'Marillion',
                    'Varys':'Varys',
                    'Hot Pie':'Hot Pie',
                    'Sansa Stark':'Sansa Stark',
                    'Leo Lefford':'Leo Lefford',
                    'All':'All', 
                    'Daenerys Targaryen':'Daenerys Targaryen', 
                    'Jorah Mormont':'Jorah Mormont', 
                    'Greatjon Umber':'Greatjon Umber',
                    'Cersei Lannister':'Cersei Baratheon', 
                    'Robb Stark':'Robb Stark', 
                    'Rickard Karstark':'Rickard Karstark',
                    'Kevan Lannister':'Kevan Lannister', 
                    'Addam Marbrand':'Addam Marbrand', 
                    'Ros':'Ros', 
                    'Sandor Clegane':'Sandor Clegane',
                    'Galbart Glover':'Galbart Glover'}

episode_ten['index'] = episode_ten['index'].replace(episode_ten_dict)

In [253]:
episode_ten = episode_ten.set_index('index')
episode_ten.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
index,,,,,,,,,,,,,,,,,,
Joffrey Baratheon,amusing,tell favor fingers tongue,fingers tongue,good,ser llyn better carry sentence,lm done day,look quite nice,grace,walk,soon youve blood lll put son,ones father,l,well,long pleases,obey,lords,None,None
Petyr Baelish,lts hard simper bow without heads,would friend found sitting,must one men city isnt man,l picture gash,must strange even years man another land despi...,carry whispering one kings ear next,useful talent lm sure youd agree,playing roles,long may reign,None,None,None,None,None,None,None,None,None
Lommy Greenhands,hes got sword one,maybe hes little squire,lets look,take,look,None,None,None,None,None,None,None,None,None,None,None,None,None
Yoren,keep mouth shut boy,youre smart boy youre trying say,youre arry hear,youve long way travel bad company,come sorry sons whores,None,None,None,None,None,None,None,None,None,None,None,None,None
Shae,father said couldnt take anyone kings landing,knew name,said dont bring shae kings landing,ashamed,im funny,None,None,None,None,None,None,None,None,None,None,None,None,None


In [254]:
episode_ten.shape

(39, 18)

In [255]:
season_one = season_one.join(episode_ten, rsuffix="_ten", how='outer')
season_one.shape

(115, 345)

In [256]:
#season_one.index

In [257]:
from numpy import nan
season_one.fillna(value=str(0), inplace=True)

In [258]:
#season_one.head(115)

In [259]:
season_one.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115 entries, Arya Stark to Galbart Glover
Columns: 345 entries, 0 to 17
dtypes: object(345)
memory usage: 310.9+ KB


In [260]:
season_one.to_pickle('/Users/zoemarkovits/Metis/project_five_got/PKLs/season_one.pkl')

## Character Breakdown

### Arya Stark

In [261]:
Arya = season_one.loc['Arya Stark']
#print Arya

In [262]:
Arya_Text = Arya.values.tolist()
#Arya_Text

In [263]:
Arya_Words = []

for word in Arya_Text:
    if len(word)>=1:
        if word != '0':
            Arya_Words.append(word)
        
#Arya_Words        

In [264]:
Arya_Words_String = ' '.join(Arya_Words)
#Arya_Words_String

In [265]:
Arya_String_Length = len(Arya_Words_String)
Arya_String_Length

1011

In [266]:
Arya_Sentiment = TextBlob(Arya_Words_String).sentiment
Arya_Sentiment

Sentiment(polarity=-0.048528138528138524, subjectivity=0.49238095238095236)

### Catelyn Stark

In [267]:
Catelyn = season_one.loc['Catelyn Stark']
#print Catelyn

In [268]:
Catelyn_Text = Catelyn.values.tolist()
#Catelyn_Text

In [269]:
Catelyn_Words = []

for word in Catelyn_Text:
    if len(word)>=1:
        if word != '0':
            Catelyn_Words.append(word)
        
#Catelyn_Words        

In [270]:
Catelyn_Words_String = ' '.join(Catelyn_Words)
#Catelyn_Words_String

In [271]:
Catelyn_String_Length = len(Catelyn_Words_String)
#Catelyn_String_Length

In [272]:
TextBlob(Catelyn_Words_String).sentiment

Sentiment(polarity=0.08284313725490197, subjectivity=0.5116946778711486)

### Cersei Lannister Baratheon

In [273]:
Cersei = season_one.loc['Cersei Baratheon']
#print Cersei

In [274]:
Cersei_Text = Cersei.values.tolist()
#Cersei_Text

In [275]:
Cersei_Words = []

for word in Cersei_Text:
    if len(word)>=1:
        if word != '0':
            Cersei_Words.append(word)
        
#Cersei_Words        

In [276]:
Cersei_Words_String = ' '.join(Cersei_Words)
#Cersei_Words_String

In [277]:
Cersei_String_Length = len(Cersei_Words_String)
Cersei_String_Length

1899

In [278]:
TextBlob(Cersei_Words_String).sentiment

Sentiment(polarity=-0.06022727272727274, subjectivity=0.47317821067821064)

### Daenerys Targaryen

In [279]:
Daenerys = season_one.loc['Daenerys Targaryen']
#print Daenerys

In [280]:
Dany_Text = Daenerys.values.tolist()
#Dany_Text

In [281]:
Dany_Words = []

for word in Dany_Text:
    if len(word)>=1:
        if word != '0':
            Dany_Words.append(word)
        
#Dany_Words        

In [282]:
Dany_Words_String = ' '.join(Dany_Words)
#Dany_Words_String

In [283]:
Dany_String_Length = len(Dany_Words_String)
Dany_String_Length

1388

In [284]:
TextBlob(Dany_Words_String).sentiment

Sentiment(polarity=0.28051948051948056, subjectivity=0.5805194805194805)

### Eddard Stark

In [285]:
Eddard = season_one.loc['Eddard Stark']
#print Eddard

In [286]:
Eddard_Text = Eddard.values.tolist()
#Eddard_Text

In [287]:
Eddard_Words = []

for word in Eddard_Text:
    if len(word)>=1:
        if word != '0':
            Eddard_Words.append(word)
        
#Eddard_Words        

In [288]:
Eddard_Words_String = ' '.join(Eddard_Words)
#Eddard_Words_String

In [289]:
Eddard_String_Length = len(Eddard_Words_String)
Eddard_String_Length

4794

In [290]:
TextBlob(Eddard_Words_String).sentiment

Sentiment(polarity=0.07185826519159849, subjectivity=0.568094837261504)

### Jaime Lannister

In [291]:
Jaime = season_one.loc['Jaime Lannister']
#print Jaime

In [292]:
Jaime_Text = Jaime.values.tolist()
#Jaime_Text

In [293]:
Jaime_Words = []

for word in Jaime_Text:
    if len(word)>=1:
        if word != '0':
            Jaime_Words.append(word)
        
#Jaime_Words        

In [294]:
Jaime_Words_String = ' '.join(Jaime_Words)
#Jaime_Words_String

In [295]:
Jaime_String_Length = len(Jaime_Words_String)
Jaime_String_Length

973

In [296]:
TextBlob(Jaime_Words_String).sentiment

Sentiment(polarity=0.012112936083524324, subjectivity=0.5282767804091334)

### Joffrey Baratheon

In [297]:
Joffrey = season_one.loc['Joffrey Baratheon']
#print Joffrey

In [298]:
Joffrey_Text = Joffrey.values.tolist()
#Joffrey_Text

In [299]:
Joffrey_Words = []

for word in Joffrey_Text:
    if len(word)>=1:
        if word != '0':
            Joffrey_Words.append(word)
        
#Joffrey_Words        

In [300]:
Joffrey_Words_String = ' '.join(Joffrey_Words)
#Joffrey_Words_String

In [301]:
Joffrey_String_Length = len(Joffrey_Words_String)
Joffrey_String_Length

769

In [302]:
TextBlob(Joffrey_Words_String).sentiment

Sentiment(polarity=0.20750000000000002, subjectivity=0.5433333333333333)

### Jon Snow

In [303]:
Jon = season_one.loc['Jon Snow']
#print Jon

In [304]:
Jon_Text = Jon.values.tolist()
#Jon_Text

In [305]:
Jon_Words = []

for word in Jon_Text:
    if len(word)>=1:
        if word != '0':
            Jon_Words.append(word)
        
#Jon_Words        

In [306]:
Jon_Words_String = ' '.join(Jon_Words)
#Jon_Words_String

In [307]:
Jon_String_Length = len(Jon_Words_String)
Jon_String_Length

1503

In [308]:
TextBlob(Jon_Words_String).sentiment

Sentiment(polarity=0.15580357142857143, subjectivity=0.49243551587301587)

### Jorah Mormont

In [309]:
Jorah = season_one.loc['Jorah Mormont']
#print Jorah

In [310]:
Jorah_Text = Jorah.values.tolist()
#Jorah_Text

In [311]:
Jorah_Words = []

for word in Jorah_Text:
    if len(word)>=1:
        if word != '0':
            Jorah_Words.append(word)
        
#Jorah_Words        

In [312]:
Jorah_Words_String = ' '.join(Jorah_Words)
#Jorah_Words_String

In [313]:
Jorah_String_Length = len(Jorah_Words_String)
Jorah_String_Length

1576

In [314]:
TextBlob(Jorah_Words_String).sentiment

Sentiment(polarity=0.0037662337662337573, subjectivity=0.4860389610389611)

### Petyr Baelish/Littlefinger

In [315]:
Littlefinger = season_one.loc['Petyr Baelish']
#print Littlefinger

In [316]:
Littlefinger_Text = Littlefinger.values.tolist()
#Littlefinger_Text

In [317]:
Littlefinger_Words = []

for word in Littlefinger_Text:
    if len(word)>=1:
        if word != '0':
            Littlefinger_Words.append(word)
        
#Littlefinger_Words        

In [318]:
Littlefinger_Words_String = ' '.join(Littlefinger_Words)
#Littlefinger_Words_String

In [319]:
Littlefinger_String_Length = len(Littlefinger_Words_String)
Littlefinger_String_Length

1797

In [320]:
TextBlob(Littlefinger_Words_String).sentiment

Sentiment(polarity=0.03150406504065041, subjectivity=0.5189121176926055)

### Robb Stark

In [321]:
Robb = season_one.loc['Robb Stark']
#print Robb

In [322]:
Robb_Text = Robb.values.tolist()
#Robb_Text

In [323]:
Robb_Words = []

for word in Robb_Text:
    if len(word)>=1:
        if word != '0':
            Robb_Words.append(word)
        
#Robb_Words        

In [324]:
Robb_Words_String = ' '.join(Robb_Words)
#Robb_Words_String

In [325]:
Robb_String_Length = len(Robb_Words_String)
#Robb_String_Length

In [326]:
TextBlob(Robb_Words_String).sentiment

Sentiment(polarity=0.2159307359307359, subjectivity=0.477900432900433)

### Samwell Tarly

In [327]:
Sam = season_one.loc['Samwell Tarly']
#print Sam

In [328]:
Sam_Text = Sam.values.tolist()
#Sam_Text

In [329]:
Sam_Words = []

for word in Sam_Text:
    if len(word)>=1:
        if word != '0':
            Sam_Words.append(word)
        
#Sam_Words        

In [330]:
Sam_Words_String = ' '.join(Sam_Words)
#Sam_Words_String

In [331]:
Sam_String_Length = len(Sam_Words_String)
Sam_String_Length

477

In [332]:
TextBlob(Sam_Words_String).sentiment

Sentiment(polarity=0.004020146520146513, subjectivity=0.5406227106227107)

### Sansa Stark

In [333]:
Sansa = season_one.loc['Sansa Stark']
#print Sansa

In [334]:
Sansa_Text = Sansa.values.tolist()
#Sansa_Text

In [335]:
Sansa_Words = []

for word in Sansa_Text:
    if len(word)>=1:
        if word != '0':
            Sansa_Words.append(word)
        
#Sansa_Words        

In [336]:
Sansa_Words_String = ' '.join(Sansa_Words)
#Sansa_Words_String

In [337]:
Sansa_String_Length = len(Sansa_Words_String)
Sansa_String_Length

944

In [338]:
TextBlob(Sansa_Words_String).sentiment

Sentiment(polarity=-0.285, subjectivity=0.6411111111111111)

### Theon Greyjoy

In [339]:
Theon = season_one.loc['Theon Greyjoy']
#print Theon

In [340]:
Theon_Text = Theon.values.tolist()
#Theon_Text

In [341]:
Theon_Words = []

for word in Theon_Text:
    if len(word)>=1:
        if word != '0':
            Theon_Words.append(word)
        
#Theon_Words        

In [342]:
Theon_Words_String = ' '.join(Theon_Words)
#Theon_Words_String

In [343]:
Theon_String_Length = len(Theon_Words_String)
Theon_String_Length

1006

In [344]:
TextBlob(Theon_Words_String).sentiment

Sentiment(polarity=0.05272486772486774, subjectivity=0.5136507936507936)

### Tyrion Lannister

In [345]:
Tyrion = season_one.loc['Tyrion Lannister']
#print Tyrion

In [346]:
Tyrion_Text = Tyrion.values.tolist()
#Tyrion_Text

In [347]:
Tyrion_Words = []

for word in Tyrion_Text:
    if len(word)>=1:
        if word != '0':
            Tyrion_Words.append(word)
        
#Tyrion_Words        

In [348]:
Tyrion_Words_String = ' '.join(Tyrion_Words)
#Tyrion_Words_String

In [349]:
Tyrion_String_Length = len(Tyrion_Words_String)
Tyrion_String_Length

3173

In [350]:
TextBlob(Tyrion_Words_String).sentiment

Sentiment(polarity=0.1536992263056093, subjectivity=0.6116934926775351)

### Tywin Lannister

In [351]:
Tywin = season_one.loc['Tywin Lannister']
#print Tywin

In [352]:
Tywin_Text = Tywin.values.tolist()
#Tywin_Text

In [353]:
Tywin_Words = []

for word in Tywin_Text:
    if len(word)>=1:
        if word != '0':
            Tywin_Words.append(word)
        
#Tywin_Words        

In [354]:
Tywin_Words_String = ' '.join(Tywin_Words)
#Tywin_Words_String

In [355]:
Tywin_String_Length = len(Tywin_Words_String)
Tywin_String_Length

693

In [356]:
TextBlob(Tywin_Words_String).sentiment

Sentiment(polarity=0.015476190476190473, subjectivity=0.5096726190476191)

### Varys

In [357]:
Varys = season_one.loc['Varys']
#print Varys

In [358]:
Varys_Text = Varys.values.tolist()
#Varys_Text

In [359]:
Varys_Words = []

for word in Varys_Text:
    if len(word)>=1:
        if word != '0':
            Varys_Words.append(word)
        
#Varys_Words        

In [360]:
Varys_Words_String = ' '.join(Varys_Words)
#Varys_Words_String

In [361]:
Varys_String_Length = len(Varys_Words_String)
Varys_String_Length

1248

In [362]:
TextBlob(Varys_Words_String).sentiment

Sentiment(polarity=0.07933273622928795, subjectivity=0.6403592576006368)